In [2]:
import pandas as pd
import requests
import re
from tqdm import tqdm
from bs4 import BeautifulSoup

In [36]:
# 地区图鉴网址
original_url = 'https://wiki.52poke.com/wiki/%E5%AE%9D%E5%8F%AF%E6%A2%A6%E5%88%97%E8%A1%A8%EF%BC%88%E6%8C%89%E6%B4%97%E7%BF%A0%E5%9B%BE%E9%89%B4%E7%BC%96%E5%8F%B7%EF%BC%89'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'
}
html = requests.get(original_url, headers=headers).text

In [50]:
# 获取信息列表
# all_data = re.findall(r'<tr>(.*?<td>.*?</td>.*?<td>.*?</td>)</tr>', html, re.S)[3:][-403:]  # 阿罗拉
all_data = re.findall(r'<tr>(.*?<td>.*?</td>.*?<td>.*?</td>)</tr>', html, re.S)[1:][:245]# 阿罗拉

# 爬取名称
name_lst = [re.findall(r'<td><a .*?title="(.*?)">', item)[-1]
            for item in all_data]

# 爬取编码信息（待转换）
numbers = [re.findall(r'<td>(#.*?)</td>', item, re.S) for item in all_data]

# 爬取属性(待转换)
tem_attributes = [re.findall(r'<a href=.*?（属性）">(.*?)</a>', item)
                  for item in all_data]

# 爬取详细网址
url_lst = ['https://wiki.52poke.com/' +
           re.search(fr'<a href="(.*?)".*?title="{item}">.*?</a>', all_data[i]).group(1)
           for i, item in enumerate(name_lst)]

# 转化为DataFrame类型
head_data = {
    '全国编号': [item[1].replace('\n', '') for item in numbers],
    '地区编号': [item[0].replace('\n', '') for item in numbers],
    '名称': name_lst,
    '属性': ['/'.join(item) for item in tem_attributes]
}
df = pd.DataFrame(head_data)

In [51]:
df

,全国编号,地区编号,名称,属性
0,#0722,#001,木木枭,草/飞行
1,#0723,#002,投羽枭,草/飞行
2,#0724,#003,狙射树枭,草/格斗
3,#0155,#004,火球鼠,火
4,#0156,#005,火岩鼠,火
...,...,...,...,...
240,#0493,#238,阿尔宙斯,一般
241,#0489,#239,霏欧纳,水
242,#0490,#240,玛纳霏,水
243,#0492,#241,谢米,草


In [54]:
# 爬取详细信息
def scrape_data(poke_url):
    poke_html = requests.get(poke_url, headers=headers).text

    # 筛选特性（普通和隐藏）
    character_text = re.search(r'>特性</a>(.*?)</tbody>', poke_html, re.S).group(1)  # 获取详细数据
    all_character_lst = re.findall(r'<a.*?title=.*?（特性）">(.*?)</a>', character_text)

    # 找出隐藏特性，若没有则None
    try:
        hidden_character_lst = re.findall(r'（特性）.*?</td>.*?title=.*?（特性）">(.*?)</a>.*?<small>', character_text, re.S)
    except AttributeError:
        hidden_character_lst = [None]

    # Pokemon普通与隐藏特性
    normal_character_lst = set(all_character_lst) - set(hidden_character_lst)  # 先移除隐藏特性
    normal_character = '/'.join(normal_character_lst)  # 普通
    hidden_character = '/'.join(hidden_character_lst)  # 隐藏

    # 暂存Pokemon的信息
    data = {
        '普通特性': normal_character,
        '隐藏特性': hidden_character,
        '类别': re.search(r'>分类<.*?title=.*?>(.*?)</a>', poke_html, re.S).group(1).strip(),
        '身高': re.search(r'>身高<.*?<td.*?>(.*?)</td>', poke_html, re.S).group(1).strip(),
        '体重': re.search(r'>体重<.*?<td.*?>(.*?)</td>', poke_html, re.S).group(1).strip(),
        '捕获率': re.search(r'>捕获率<.*?<td.*?>(.*?)<small>', poke_html, re.S).group(1).strip(),
        # HP, 攻击, 防御, 特攻, 特防, 速度, 总和
        '种族值': re.findall(r'<span style="float:right">(.*?)</span>', poke_html)[:7]
    }
    return data


# 爬取第二部分信息
# scrape_data(url_lst[200])
body_data = [scrape_data(url) for url in tqdm(url_lst)]

100%|██████████| 245/245 [02:34<00:00,  1.59it/s]


In [55]:
# 拼接数据
poke_data = pd.concat([pd.DataFrame(head_data), pd.DataFrame(body_data)], axis=1)

# 保存为csv
poke_data.to_csv(r'd:\desktop\洗翠图鉴.csv', encoding='utf-8-sig', index=False)